In [1]:
# Data manipulation
from sklearn.preprocessing import LabelEncoder
from scipy.stats import loguniform
import numpy as np
import pandas as pd
import os

# Data Visualazation
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme()
%matplotlib inline

#consistent sized plots
from pylab import rcParams
rcParams['figure.figsize'] = 12,5
rcParams['xtick.labelsize'] = 12
rcParams['ytick.labelsize'] = 12
rcParams['axes.titlesize'] = 14
sns.set_style("darkgrid", {"grid.color": ".6", "grid.linestyle": ":"})

#handle unwanted warnings
import warnings
warnings.filterwarnings('ignore')

# Machine learning
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
from lightgbm import LGBMModel,LGBMClassifier
import catboost as cb
from catboost import CatBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.ensemble import VotingClassifier

# Model Evaluations
import sklearn as sk
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import accuracy_score

#no constraint on the data columns to be displayed in jupyter notebook
pd.options.display.max_columns = None

# Handle Date & Time
from datetime import datetime

# Handle zip files
import zipfile

# Print module versions for reproducibility
print('CatBoost version {}'.format(cb.__version__))
print('NumPy version {}'.format(np.__version__))
print('Pandas version {}'.format(pd.__version__))
print('seaborn version {}'.format(sns.__version__))
print('sklearn version {}'.format(sk.__version__))
print('xgboost version {}'.format(xgb.__version__))
print('lightgbm version {}'.format(lgb.__version__))

# User Defined Functions

# a) BoxPlot:- To check Outliers
def Create_BoxPlotS(DataFrame, Features, Rows, Columns):
    fig=plt.figure(figsize=(20,20))
    for i, Feature in enumerate(Features[0:]):
        ax=fig.add_subplot(Rows,Columns,i+1)
        sns.boxplot(DataFrame[Feature],ax=ax)   
    fig.tight_layout()  
    plt.show()
    
# b) DistPlot:- To check Normality
def Create_DistPlots(DataFrame, Features, Rows, Columns):
    fig=plt.figure(figsize=(20,20))
    for i, feature in enumerate(Features[0:]):
        ax=fig.add_subplot(Rows,Columns,i+1)
        sns.distplot(DataFrame[feature],ax=ax)    
    fig.tight_layout()  
    plt.show()

# c) Univariate Analysis of Catagorical Variables
def Univariate_Cat_Features(DataFrame, Features, Rows, Columns):
    fig=plt.figure(figsize=(20,20))
    for i, feature in enumerate(Features[0:]):
        ax=fig.add_subplot(Rows,Columns,i+1)
        sns.countplot(DataFrame[feature],ax=ax)    
    fig.tight_layout()  
    plt.show()
    
# d) Scatter Plot:- Bivariate Analysis between Target Variable & Independent Variables
# Note:- Pass Target_Variable in Single Quote in this function
def Create_ScatterPlots(DataFrame, Features, Target_Variable, Rows, Columns):
    fig=plt.figure(figsize=(20,20))
    for i, feature in enumerate(Features[0:]):
        ax=fig.add_subplot(Rows,Columns,i+1)
        sns.scatterplot(DataFrame[feature],DataFrame[Target_Variable],ax=ax)  
    fig.tight_layout()  
    plt.show()

# e) Joint Plot:- Bivariate Analysis between Target Variable & Independent Variables
# Note:- Pass Target_Variable in Single Quote in this function
def Create_JointPlots(DataFrame, Features, Target_Variable):
    fig=plt.figure(figsize=(20,20))
    for i, feature in enumerate(Features[0:]):
        sns.jointplot(y= Target_Variable, x= DataFrame[feature], data = DataFrame,kind='reg')
    fig.tight_layout()  
    plt.show()

# f) Check Feature wise Outliers   
def Check_Outliers(DataFrame, Column_List):
    for column in Column_List:    
        q1 = DataFrame[column].quantile(0.25)    # First Quartile
        q3 = DataFrame[column].quantile(0.75)    # Third Quartile
        IQR = q3 - q1                            # Inter Quartile Range
        llimit = q1 - 1.5*IQR                    # Lower Limit
        ulimit = q3 + 1.5*IQR                    # Upper Limit
        outliers = DataFrame[(DataFrame[column] < llimit) | (DataFrame[column] > ulimit)]
        print('\nNumber of outliers in "' + column + '" :' + str(len(outliers)))
        print('\nLower Limit in "' + column + '" :',llimit)
        print('\nUpper Limit in "' + column + '" :',ulimit)
        print('\nInter Quartile Range of "' + column + '" :',IQR,"\n")

# g) Label Encoding   
def Label_Encoder(DataFrame):
    le = LabelEncoder()
    for col in DataFrame.columns:
        if(DataFrame[col].dtype == 'object'):
            DataFrame.loc[:,col] = le.fit_transform(DataFrame.loc[:,col])
            
            
train = pd.read_csv (r'C:\Users\1014070\Favorites\AB\TG\HCP-June23\Train.csv',encoding='latin-1')
test = pd.read_csv (r'C:\Users\1014070\Favorites\AB\TG\HCP-June23\Test.csv',encoding='latin-1')
test1 = pd.read_csv (r'C:\Users\1014070\Favorites\AB\TG\HCP-June23\Test.csv',encoding='latin-1')
submission = pd.read_csv (r'C:\Users\1014070\Favorites\AB\TG\HCP-June23\Sample_Submission.csv',encoding='latin-1')


train['USERCITY'] = train['USERCITY'].fillna(train['USERCITY'].mode()[0],inplace=True)
test['USERCITY'] = test['USERCITY'].fillna(test['USERCITY'].mode()[0],inplace=True)
train['USERZIPCODE'] = train['USERZIPCODE'].fillna(train['USERZIPCODE'].mode()[0],inplace=True)
test['USERZIPCODE'] = test['USERZIPCODE'].fillna(test['USERZIPCODE'].mode()[0],inplace=True)
train['USERAGENT'] = train['USERAGENT'].fillna(train['USERAGENT'].mode()[0],inplace=True)
test['USERAGENT'] = test['USERAGENT'].fillna(test['USERAGENT'].mode()[0],inplace=True)

train.loc[(train['IS_HCP'].isnull()),'IS_HCP'] = 0

Label_Encoder(train)
Label_Encoder(test)

CatBoost version 1.2
NumPy version 1.21.5
Pandas version 1.4.2
seaborn version 0.11.2
sklearn version 1.0.2
xgboost version 1.7.6
lightgbm version 3.3.5


In [2]:
#train = train.drop(['TAXONOMY'],axis=1) #S1

##############################  S2     #############################
# train = train.drop(['TAXONOMY','URL'],axis=1)

# test = test.drop(['URL'],axis=1)


##############################  S3    #############################

# train = train.drop(['TAXONOMY','URL','USERAGENT'],axis=1)

# test = test.drop(['URL','USERAGENT'],axis=1)

##############################  S4    #############################

# train = train.drop(['TAXONOMY','URL','USERPLATFORMUID'],axis=1)

# test = test.drop(['URL','USERPLATFORMUID'],axis=1)

##############################  S5    #############################

# train = train.drop(['DEVICETYPE','PLATFORM_ID','BIDREQUESTIP'
#                     ,'USERCITY','USERZIPCODE','USERAGENT'
#                     ,'PLATFORMTYPE','CHANNELTYPE','TAXONOMY'
#                     ,'URL','USERPLATFORMUID'],axis=1)

# test = test.drop(['DEVICETYPE','PLATFORM_ID','BIDREQUESTIP'
#                     ,'USERCITY','USERZIPCODE','USERAGENT'
#                     ,'PLATFORMTYPE','CHANNELTYPE'
#                     ,'URL','USERPLATFORMUID'],axis=1)

# THIS IS BASE REFERENCE
##############################  S6    #############################

train = train.drop(['PLATFORM_ID','BIDREQUESTIP'
                    ,'USERCITY','USERZIPCODE','USERAGENT'
                    ,'PLATFORMTYPE','CHANNELTYPE','TAXONOMY'
                    ,'URL','USERPLATFORMUID'],axis=1)

test = test.drop(['PLATFORM_ID','BIDREQUESTIP'
                    ,'USERCITY','USERZIPCODE','USERAGENT'
                    ,'PLATFORMTYPE','CHANNELTYPE'
                    ,'URL','USERPLATFORMUID'],axis=1)

#DEVICETYPE:- Not Giving much impact

##############################  S7    #############################

train = train.drop(['BIDREQUESTIP'
                    ,'USERCITY','USERZIPCODE','USERAGENT'
                    ,'PLATFORMTYPE','CHANNELTYPE','TAXONOMY'
                    ,'URL','USERPLATFORMUID'],axis=1)

test = test.drop(['BIDREQUESTIP'
                    ,'USERCITY','USERZIPCODE','USERAGENT'
                    ,'PLATFORMTYPE','CHANNELTYPE'
                    ,'URL','USERPLATFORMUID'],axis=1)

In [3]:
X = train.drop(['IS_HCP'],axis=1)
y = train.IS_HCP
X_train, X_test, y_train, y_test = train_test_split(X,y, shuffle = True)

In [14]:
#model5 = LogisticRegression(random_state=42) #46.4611 S1
#model5 = LogisticRegression(random_state=42) #41.277 S2
#model5 = LogisticRegression(random_state=42) #41.277 S3
#model5 = LogisticRegression(random_state=42) #41.277 S4
#model5 = LogisticRegression(random_state=42) #41.277 S5

model5 = LogisticRegression(random_state=42) #41.277 S6

model5.fit(X_train,y_train) 
print('Accuracy of Logistic Regression on training set: {:.2f}'
     .format(model5.score(X_train, y_train)))

print('Accuracy of Logistic Regression on test set: {:.2f}'
     .format(model5.score(X_test, y_test)))
print('-------------'*9)
print('Classification Report of the training data:\n\n',classification_report(y_test,model5.predict(X_test)),'\n')

# Import Test data for the prediction of the Target Variable
x = np.array(test)

# Prediction using above Tuned Mode

y_pred5 = model5.predict(x); 
y_pred = y_pred5

# Save in Dataframe
df1=pd.DataFrame(y_pred,columns=['IS_HCP']); print(type(df1))

submission.head(); submission1 = submission.drop(['IS_HCP'], axis=1); submission.head()
final_pred = pd.concat([submission1,df1], axis=1); final_pred.head()
final_pred.to_csv(r'C:\Users\1014070\Favorites\AB\TG\HCP-June23\M5.csv',index=False)
print("Process Completed")

Accuracy of Logistic Regression on training set: 0.70
Accuracy of Logistic Regression on test set: 0.70
---------------------------------------------------------------------------------------------------------------------
Classification Report of the training data:

               precision    recall  f1-score   support

         0.0       0.70      1.00      0.82     19950
         1.0       0.00      0.00      0.00      8535

    accuracy                           0.70     28485
   macro avg       0.35      0.50      0.41     28485
weighted avg       0.49      0.70      0.58     28485
 

<class 'pandas.core.frame.DataFrame'>
Process Completed


In [75]:
# coefficients = model5.coef_[0]

# feature_importance = pd.DataFrame({'Feature': X.columns, 'Importance': np.abs(coefficients)})
# feature_importance = feature_importance.sort_values('Importance', ascending=True)
# feature_importance.plot(x='Feature', y='Importance', kind='barh', figsize=(10, 6))

In [15]:
# Hyper Version
# model55 = LogisticRegression(random_state=42
#                             ,solver = 'newton-cg', penalty = 'l2'
#                              , C = 0.1) # 48.4646 S1

# model55 = LogisticRegression(random_state=42
#                             ,solver = 'newton-cg', penalty = 'l2'
#                              , C = 0.1) # 49.9644 S2

# model55 = LogisticRegression(random_state=42
#                             ,solver = 'newton-cg', penalty = 'l2'
#                              , C = 0.1) #49.3564 S3

# model55 = LogisticRegression(random_state=42
#                             ,solver = 'newton-cg', penalty = 'l2'
#                              , C = 0.1) #48.9298 S4

# model55 = LogisticRegression(random_state=42
#                             ,solver = 'newton-cg', penalty = 'l2'
#                              , C = 0.1) #41.277 S5

model55 = LogisticRegression(random_state=42
                            ,solver = 'newton-cg', penalty = 'l2'
                             , C = 0.1) #41.277 S6
model55.fit(X_train,y_train) 
print('Accuracy of Logistic Regression on training set: {:.2f}'
     .format(model55.score(X_train, y_train)))

print('Accuracy of Logistic Regression on test set: {:.2f}'
     .format(model55.score(X_test, y_test)))
print('-------------'*9)
print('Classification Report of the training data:\n\n',classification_report(y_test,model55.predict(X_test)),'\n')

# Import Test data for the prediction of the Target Variable
x = np.array(test)

# Prediction using above Tuned Mode

y_pred55 = model55.predict(x); 
y_pred = y_pred55

# Save in Dataframe
df1=pd.DataFrame(y_pred,columns=['IS_HCP']); print(type(df1))

submission.head(); submission1 = submission.drop(['IS_HCP'], axis=1); submission.head()
final_pred = pd.concat([submission1,df1], axis=1); final_pred.head()
final_pred.to_csv(r'C:\Users\1014070\Favorites\AB\TG\HCP-June23\M55.csv',index=False)
print("Process Completed")

Accuracy of Logistic Regression on training set: 0.70
Accuracy of Logistic Regression on test set: 0.70
---------------------------------------------------------------------------------------------------------------------
Classification Report of the training data:

               precision    recall  f1-score   support

         0.0       0.70      1.00      0.82     19950
         1.0       0.00      0.00      0.00      8535

    accuracy                           0.70     28485
   macro avg       0.35      0.50      0.41     28485
weighted avg       0.49      0.70      0.58     28485
 

<class 'pandas.core.frame.DataFrame'>
Process Completed


In [16]:
#model1 = RandomForestClassifier(random_state=0) #43.0604 S1
#model1 = RandomForestClassifier(random_state=0) #53.1873 S2
#model1 = RandomForestClassifier(random_state=0) #50.0546 S3
#model1 = RandomForestClassifier(random_state=0) #45.5455 S4
#model1 = RandomForestClassifier(random_state=0) #29.1222 S5

model1 = RandomForestClassifier(random_state=0) #28.2487 S6
model1.fit(X_train,y_train) 
print('Accuracy of Random Forest Classifier on training set: {:.2f}'
     .format(model1.score(X_train, y_train)))

print('Accuracy of Random Forest Classifier on test set: {:.2f}'
     .format(model1.score(X_test, y_test)))
print('-------------'*9)
print('Classification Report of the training data:\n\n',classification_report(y_test,model1.predict(X_test)),'\n')

# Import Test data for the prediction of the Target Variable
x = np.array(test)

# Prediction using above Tuned Mode

y_pred1 = model1.predict(x); 
y_pred = y_pred1

# Save in Dataframe
df1=pd.DataFrame(y_pred,columns=['IS_HCP']); print(type(df1))

submission.head(); submission1 = submission.drop(['IS_HCP'], axis=1); submission.head()
final_pred = pd.concat([submission1,df1], axis=1); final_pred.head()
final_pred.to_csv(r'C:\Users\1014070\Favorites\AB\TG\HCP-June23\M1.csv',index=False)
print("Process Completed")

Accuracy of Random Forest Classifier on training set: 1.00
Accuracy of Random Forest Classifier on test set: 0.96
---------------------------------------------------------------------------------------------------------------------
Classification Report of the training data:

               precision    recall  f1-score   support

         0.0       0.96      0.99      0.97     19950
         1.0       0.97      0.91      0.94      8535

    accuracy                           0.96     28485
   macro avg       0.97      0.95      0.96     28485
weighted avg       0.96      0.96      0.96     28485
 

<class 'pandas.core.frame.DataFrame'>
Process Completed


In [17]:
# Hyper Version
# model11 = RandomForestClassifier(random_state=0
#                                ,n_estimators = 50, min_samples_split = 2
#                                 ,min_samples_leaf = 3, max_features = 'auto'
#                                 , max_depth = 10, bootstrap =  False) #56.4211 S1

# model11 = RandomForestClassifier(random_state=0
#                                ,n_estimators = 50, min_samples_split = 2
#                                 ,min_samples_leaf = 3, max_features = 'auto'
#                                 , max_depth = 10, bootstrap =  False) #62.6583 S2

# model11 = RandomForestClassifier(random_state=0
#                                ,n_estimators = 50, min_samples_split = 2
#                                 ,min_samples_leaf = 3, max_features = 'auto'
#                                 , max_depth = 10, bootstrap =  False) #62.8263 S3

# model11 = RandomForestClassifier(random_state=0
#                                ,n_estimators = 50, min_samples_split = 2
#                                 ,min_samples_leaf = 3, max_features = 'auto'
#                                 , max_depth = 10, bootstrap =  False) #60.1449 S4

# model11 = RandomForestClassifier(random_state=0
#                                ,n_estimators = 50, min_samples_split = 2
#                                 ,min_samples_leaf = 3, max_features = 'auto'
#                                 , max_depth = 10, bootstrap =  False) #34.8835 S5

model11 = RandomForestClassifier(random_state=0
                               ,n_estimators = 50, min_samples_split = 2
                                ,min_samples_leaf = 3, max_features = 'auto'
                                , max_depth = 10, bootstrap =  False) #29.8672 S6
model11.fit(X_train,y_train) 
print('Accuracy of Random Forest Classifier on training set: {:.2f}'
     .format(model11.score(X_train, y_train)))

print('Accuracy of Random Forest Classifier on test set: {:.2f}'
     .format(model11.score(X_test, y_test)))
print('-------------'*9)
print('Classification Report of the training data:\n\n',classification_report(y_test,model11.predict(X_test)),'\n')

# Import Test data for the prediction of the Target Variable
x = np.array(test)

# Prediction using above Tuned Mode

y_pred11 = model11.predict(x); 
y_pred = y_pred11

# Save in Dataframe
df1=pd.DataFrame(y_pred,columns=['IS_HCP']); print(type(df1))

submission.head(); submission1 = submission.drop(['IS_HCP'], axis=1); submission.head()
final_pred = pd.concat([submission1,df1], axis=1); final_pred.head()
final_pred.to_csv(r'C:\Users\1014070\Favorites\AB\TG\HCP-June23\M11.csv',index=False)
print("Process Completed")

Accuracy of Random Forest Classifier on training set: 0.81
Accuracy of Random Forest Classifier on test set: 0.81
---------------------------------------------------------------------------------------------------------------------
Classification Report of the training data:

               precision    recall  f1-score   support

         0.0       0.79      0.99      0.88     19950
         1.0       0.94      0.38      0.54      8535

    accuracy                           0.81     28485
   macro avg       0.86      0.68      0.71     28485
weighted avg       0.83      0.81      0.78     28485
 

<class 'pandas.core.frame.DataFrame'>
Process Completed


In [19]:
#model2 = LGBMClassifier(random_state=0) #50.0098 S1
#model2 = LGBMClassifier(random_state=0) #59.2875 S2
#model2 = LGBMClassifier(random_state=0) #57.5435 S3
#model2 = LGBMClassifier(random_state=0) #55.9512 S4
#model2 = LGBMClassifier(random_state=0) #59.1254 S5


model2 = LGBMClassifier(random_state=0) #59.1254 S6
model2.fit(X_train,y_train) 
print('Accuracy of LGBM Classifier on training set: {:.2f}'
     .format(model2.score(X_train, y_train)))

print('Accuracy of LGBM Classifier on test set: {:.2f}'
     .format(model2.score(X_test, y_test)))
print('-------------'*9)
print('Classification Report of the training data:\n\n',classification_report(y_test,model2.predict(X_test)),'\n')

# Import Test data for the prediction of the Target Variable
x = np.array(test)

# Prediction using above Tuned Mode

y_pred2 = model2.predict(x); 
y_pred = y_pred2

# Save in Dataframe
df1=pd.DataFrame(y_pred,columns=['IS_HCP']); print(type(df1))

submission.head(); submission1 = submission.drop(['IS_HCP'], axis=1); submission.head()
final_pred = pd.concat([submission1,df1], axis=1); final_pred.head()
final_pred.to_csv(r'C:\Users\1014070\Favorites\AB\TG\HCP-June23\M2.csv',index=False)
print("Process Completed")

Accuracy of LGBM Classifier on training set: 0.90
Accuracy of LGBM Classifier on test set: 0.90
---------------------------------------------------------------------------------------------------------------------
Classification Report of the training data:

               precision    recall  f1-score   support

         0.0       0.91      0.96      0.93     19950
         1.0       0.88      0.78      0.83      8535

    accuracy                           0.90     28485
   macro avg       0.90      0.87      0.88     28485
weighted avg       0.90      0.90      0.90     28485
 

<class 'pandas.core.frame.DataFrame'>
Process Completed


In [20]:
# Hyper Version
# model22 = LGBMClassifier(random_state=42,reg_alpha = 2,num_leaves = 50
#                          , n_estimators = 300, min_child_samples = 40
#                          , max_depth = -1, learning_rate = 0.3, colsample_bytree = 0.5
#                          , boosting_type = 'dart') #51.3348 S1

# model22 = LGBMClassifier(random_state=42,reg_alpha = 2,num_leaves = 50
#                          , n_estimators = 300, min_child_samples = 40
#                          , max_depth = -1, learning_rate = 0.3, colsample_bytree = 0.5
#                          , boosting_type = 'dart') #52.3449 S2

# model22 = LGBMClassifier(random_state=42,reg_alpha = 2,num_leaves = 50
#                          , n_estimators = 300, min_child_samples = 40
#                          , max_depth = -1, learning_rate = 0.3, colsample_bytree = 0.5
#                          , boosting_type = 'dart') #50.0667 S3

# model22 = LGBMClassifier(random_state=42,reg_alpha = 2,num_leaves = 50
#                          , n_estimators = 300, min_child_samples = 40
#                          , max_depth = -1, learning_rate = 0.3, colsample_bytree = 0.5
#                          , boosting_type = 'dart') #24.7871 S5

model22 = LGBMClassifier(random_state=42,reg_alpha = 2,num_leaves = 50
                         , n_estimators = 300, min_child_samples = 40
                         , max_depth = -1, learning_rate = 0.3, colsample_bytree = 0.5
                         , boosting_type = 'dart') #24.7871 S6
model22.fit(X_train,y_train) 
print('Accuracy of LGBM Classifier on training set: {:.2f}'
     .format(model22.score(X_train, y_train)))

print('Accuracy of LGBM Classifier on test set: {:.2f}'
     .format(model22.score(X_test, y_test)))
print('-------------'*9)
print('Classification Report of the training data:\n\n',classification_report(y_test,model22.predict(X_test)),'\n')

# Import Test data for the prediction of the Target Variable
x = np.array(test)

# Prediction using above Tuned Mode

y_pred22 = model22.predict(x); 
y_pred = y_pred22

# Save in Dataframe
df1=pd.DataFrame(y_pred,columns=['IS_HCP']); print(type(df1))

submission.head(); submission1 = submission.drop(['IS_HCP'], axis=1); submission.head()
final_pred = pd.concat([submission1,df1], axis=1); final_pred.head()
final_pred.to_csv(r'C:\Users\1014070\Favorites\AB\TG\HCP-June23\M22.csv',index=False)
print("Process Completed")

Accuracy of LGBM Classifier on training set: 0.51
Accuracy of LGBM Classifier on test set: 0.51
---------------------------------------------------------------------------------------------------------------------
Classification Report of the training data:

               precision    recall  f1-score   support

         0.0       0.96      0.31      0.47     19950
         1.0       0.38      0.97      0.54      8535

    accuracy                           0.51     28485
   macro avg       0.67      0.64      0.51     28485
weighted avg       0.78      0.51      0.49     28485
 

<class 'pandas.core.frame.DataFrame'>
Process Completed


In [21]:
#model3 = CatBoostClassifier(random_state=0, logging_level='Silent') #44.6453 S1
#model3 = CatBoostClassifier(random_state=0, logging_level='Silent') #49.9428 S2
#model3 = CatBoostClassifier(random_state=0, logging_level='Silent') #49.4647 S3
#model3 = CatBoostClassifier(random_state=0, logging_level='Silent') #48.8649 S5

model3 = CatBoostClassifier(random_state=0, logging_level='Silent') #48.8649 S6
model3.fit(X_train,y_train) 
print('Accuracy of CatBoost Classifier on training set: {:.2f}'
     .format(model3.score(X_train, y_train)))

print('Accuracy of CatBoost Classifier on test set: {:.2f}'
     .format(model3.score(X_test, y_test)))
print('-------------'*9)
print('Classification Report of the training data:\n\n',classification_report(y_test,model3.predict(X_test)),'\n')

# Import Test data for the prediction of the Target Variable
x = np.array(test)

# Prediction using above Tuned Mode

y_pred3 = model3.predict(x); 
y_pred = y_pred3

# Save in Dataframe
df1=pd.DataFrame(y_pred,columns=['IS_HCP']); print(type(df1))

submission.head(); submission1 = submission.drop(['IS_HCP'], axis=1); submission.head()
final_pred = pd.concat([submission1,df1], axis=1); final_pred.head()
final_pred.to_csv(r'C:\Users\1014070\Favorites\AB\TG\HCP-June23\M3.csv',index=False)
print("Process Completed")

Accuracy of CatBoost Classifier on training set: 0.92
Accuracy of CatBoost Classifier on test set: 0.92
---------------------------------------------------------------------------------------------------------------------
Classification Report of the training data:

               precision    recall  f1-score   support

         0.0       0.93      0.95      0.94     19950
         1.0       0.88      0.84      0.86      8535

    accuracy                           0.92     28485
   macro avg       0.91      0.89      0.90     28485
weighted avg       0.92      0.92      0.92     28485
 

<class 'pandas.core.frame.DataFrame'>
Process Completed


In [22]:
#Hyper Version
# model33 = CatBoostClassifier(random_state=0
#                             , depth = 10, iterations = 100, learning_rate = 0.04,logging_level='Silent') #55.0339 S1

# model33 = CatBoostClassifier(random_state=0
#                             , depth = 10, iterations = 100, learning_rate = 0.04,logging_level='Silent') #60.5592 S2

# model33 = CatBoostClassifier(random_state=0
#                             , depth = 10, iterations = 100, learning_rate = 0.04,logging_level='Silent') #58.6436 S3

# model33 = CatBoostClassifier(random_state=0
#                             , depth = 10, iterations = 100, learning_rate = 0.04,logging_level='Silent') #54.692 S5

model33 = CatBoostClassifier(random_state=0
                            , depth = 10, iterations = 100, learning_rate = 0.04,logging_level='Silent') #54.692 S6
model33.fit(X_train,y_train) 
print('Accuracy of CatBoost Classifier on training set: {:.2f}'
     .format(model33.score(X_train, y_train)))

print('Accuracy of CatBoost Classifier on test set: {:.2f}'
     .format(model33.score(X_test, y_test)))
print('-------------'*9)
print('Classification Report of the training data:\n\n',classification_report(y_test,model33.predict(X_test)),'\n')

# Import Test data for the prediction of the Target Variable
x = np.array(test)

# Prediction using above Tuned Mode

y_pred33 = model33.predict(x); 
y_pred = y_pred33

# Save in Dataframe
df1=pd.DataFrame(y_pred,columns=['IS_HCP']); print(type(df1))

submission.head(); submission1 = submission.drop(['IS_HCP'], axis=1); submission.head()
final_pred = pd.concat([submission1,df1], axis=1); final_pred.head()
final_pred.to_csv(r'C:\Users\1014070\Favorites\AB\TG\HCP-June23\M33.csv',index=False)
print("Process Completed")

Accuracy of CatBoost Classifier on training set: 0.85
Accuracy of CatBoost Classifier on test set: 0.85
---------------------------------------------------------------------------------------------------------------------
Classification Report of the training data:

               precision    recall  f1-score   support

         0.0       0.84      0.98      0.90     19950
         1.0       0.92      0.57      0.70      8535

    accuracy                           0.85     28485
   macro avg       0.88      0.77      0.80     28485
weighted avg       0.86      0.85      0.84     28485
 

<class 'pandas.core.frame.DataFrame'>
Process Completed


In [23]:
#model4 = XGBClassifier(random_state=0) #48.87 S1
#model4 = XGBClassifier(random_state=0) #58.535 S2
#model4 = XGBClassifier(random_state=0) #56.5341 S3
#model4 = XGBClassifier(random_state=0) #57.838 S5

model4 = XGBClassifier(random_state=0) #57.838 S6

model4.fit(X_train,y_train) 
print('Accuracy of XGB Classifier on training set: {:.2f}'
     .format(model4.score(X_train, y_train)))

print('Accuracy of XGB Classifier on test set: {:.2f}'
     .format(model4.score(X_test, y_test)))
print('-------------'*9)
print('Classification Report of the training data:\n\n',classification_report(y_test,model4.predict(X_test)),'\n')

# Import Test data for the prediction of the Target Variable
x = np.array(test)

# Prediction using above Tuned Mode

y_pred4 = model4.predict(x); 
y_pred = y_pred4

# Save in Dataframe
df1=pd.DataFrame(y_pred,columns=['IS_HCP']); print(type(df1))

submission.head(); submission1 = submission.drop(['IS_HCP'], axis=1); submission.head()
final_pred = pd.concat([submission1,df1], axis=1); final_pred.head()
final_pred.to_csv(r'C:\Users\1014070\Favorites\AB\TG\HCP-June23\M4.csv',index=False)
print("Process Completed")

Accuracy of XGB Classifier on training set: 0.98
Accuracy of XGB Classifier on test set: 0.98
---------------------------------------------------------------------------------------------------------------------
Classification Report of the training data:

               precision    recall  f1-score   support

         0.0       0.98      0.99      0.99     19950
         1.0       0.98      0.96      0.97      8535

    accuracy                           0.98     28485
   macro avg       0.98      0.97      0.98     28485
weighted avg       0.98      0.98      0.98     28485
 

<class 'pandas.core.frame.DataFrame'>
Process Completed


In [10]:
# Hyper Version XGBClassifier


In [9]:
#model6 = KNeighborsClassifier(n_jobs= -1) #48.538 S1

#model6 = KNeighborsClassifier(n_jobs= -1) #49.7868 S2
#model6 = KNeighborsClassifier(n_jobs= -1) #47.5075 S3

model6 = KNeighborsClassifier(n_jobs= -1) #33.7273 S5
model6.fit(X_train,y_train) 
print('Accuracy of KNeighbors Classifier on training set: {:.2f}'
     .format(model6.score(X_train, y_train)))

print('Accuracy of KNeighbors Classifier on test set: {:.2f}'
     .format(model6.score(X_test, y_test)))
print('-------------'*9)
print('Classification Report of the training data:\n\n',classification_report(y_test,model6.predict(X_test)),'\n')

# Import Test data for the prediction of the Target Variable
x = np.array(test)

# Prediction using above Tuned Mode

y_pred6 = model6.predict(x); 
y_pred = y_pred6

# Save in Dataframe
df1=pd.DataFrame(y_pred,columns=['IS_HCP']); print(type(df1))

submission.head(); submission1 = submission.drop(['IS_HCP'], axis=1); submission.head()
final_pred = pd.concat([submission1,df1], axis=1); final_pred.head()
final_pred.to_csv(r'C:\Users\1014070\Favorites\AB\TG\HCP-June23\M6.csv',index=False)
print("Process Completed")

Accuracy of KNeighbors Classifier on training set: 0.77
Accuracy of KNeighbors Classifier on test set: 0.67
---------------------------------------------------------------------------------------------------------------------
Classification Report of the training data:

               precision    recall  f1-score   support

         0.0       0.73      0.83      0.78     19950
         1.0       0.43      0.30      0.35      8535

    accuracy                           0.67     28485
   macro avg       0.58      0.56      0.57     28485
weighted avg       0.64      0.67      0.65     28485
 

<class 'pandas.core.frame.DataFrame'>
Process Completed


In [10]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
#model7 = make_pipeline(StandardScaler(),LinearSVC()) #46.8387 S1
#model7 = make_pipeline(StandardScaler(),LinearSVC()) #47.2893 S2
#model7 = make_pipeline(StandardScaler(),LinearSVC()) #47.1324 S3

model7 = make_pipeline(StandardScaler(),LinearSVC()) #41.277 S5
model7.fit(X_train,y_train) 
print('Accuracy of Linear SVC on training set: {:.2f}'
     .format(model7.score(X_train, y_train)))

print('Accuracy of Linear SVC on test set: {:.2f}'
     .format(model7.score(X_test, y_test)))
print('-------------'*9)
print('Classification Report of the training data:\n\n',classification_report(y_test,model7.predict(X_test)),'\n')

# Import Test data for the prediction of the Target Variable
x = np.array(test)

# Prediction using above Tuned Mode

y_pred7 = model7.predict(x); 
y_pred = y_pred7

# Save in Dataframe
df1=pd.DataFrame(y_pred,columns=['IS_HCP']); print(type(df1))

submission.head(); submission1 = submission.drop(['IS_HCP'], axis=1); submission.head()
final_pred = pd.concat([submission1,df1], axis=1); final_pred.head()
final_pred.to_csv(r'C:\Users\1014070\Favorites\AB\TG\HCP-June23\M7.csv',index=False)
print("Process Completed")

Accuracy of Linear SVC on training set: 0.70
Accuracy of Linear SVC on test set: 0.70
---------------------------------------------------------------------------------------------------------------------
Classification Report of the training data:

               precision    recall  f1-score   support

         0.0       0.70      1.00      0.82     19950
         1.0       0.00      0.00      0.00      8535

    accuracy                           0.70     28485
   macro avg       0.35      0.50      0.41     28485
weighted avg       0.49      0.70      0.58     28485
 

<class 'pandas.core.frame.DataFrame'>
Process Completed


In [11]:
#model8 = make_pipeline(StandardScaler(),SGDClassifier()) #45.5557 S1
#model8 = make_pipeline(StandardScaler(),SGDClassifier()) #43.6636 S2
#model8 = make_pipeline(StandardScaler(),SGDClassifier()) #41.277 S3

model8 = make_pipeline(StandardScaler(),SGDClassifier()) #41.277 S5


model8.fit(X_train,y_train) 
print('Accuracy of SGD Classifier on training set: {:.2f}'
     .format(model8.score(X_train, y_train)))

print('Accuracy of SGD Classifier on test set: {:.2f}'
     .format(model8.score(X_test, y_test)))
print('-------------'*9)
print('Classification Report of the training data:\n\n',classification_report(y_test,model8.predict(X_test)),'\n')

# Import Test data for the prediction of the Target Variable
x = np.array(test)

# Prediction using above Tuned Mode

y_pred8 = model8.predict(x); 
y_pred = y_pred8

# Save in Dataframe
df1=pd.DataFrame(y_pred,columns=['IS_HCP']); print(type(df1))

submission.head(); submission1 = submission.drop(['IS_HCP'], axis=1); submission.head()
final_pred = pd.concat([submission1,df1], axis=1); final_pred.head()
final_pred.to_csv(r'C:\Users\1014070\Favorites\AB\TG\HCP-June23\M8.csv',index=False)
print("Process Completed")

Accuracy of SGD Classifier on training set: 0.70
Accuracy of SGD Classifier on test set: 0.70
---------------------------------------------------------------------------------------------------------------------
Classification Report of the training data:

               precision    recall  f1-score   support

         0.0       0.70      1.00      0.82     19950
         1.0       0.00      0.00      0.00      8535

    accuracy                           0.70     28485
   macro avg       0.35      0.50      0.41     28485
weighted avg       0.49      0.70      0.58     28485
 

<class 'pandas.core.frame.DataFrame'>
Process Completed


In [12]:
mm = make_pipeline(StandardScaler(),LinearSVC()) 
#model9 = CalibratedClassifierCV(mm) #47.341 S1
#model9 = CalibratedClassifierCV(mm) #50.1622 S2
#model9 = CalibratedClassifierCV(mm) #49.361 S3

model9 = CalibratedClassifierCV(mm) #41.277 S5

model9.fit(X_train,y_train) 
print('Accuracy of Linear SVC on training set: {:.2f}'
     .format(model9.score(X_train, y_train)))

print('Accuracy of Linear SVC on test set: {:.2f}'
     .format(model9.score(X_test, y_test)))
print('-------------'*9)
print('Classification Report of the training data:\n\n',classification_report(y_test,model9.predict(X_test)),'\n')

# Import Test data for the prediction of the Target Variable
x = np.array(test)

# Prediction using above Tuned Mode

y_pred9 = model9.predict(x); 
y_pred = y_pred9

# Save in Dataframe
df1=pd.DataFrame(y_pred,columns=['IS_HCP']); print(type(df1))

submission.head(); submission1 = submission.drop(['IS_HCP'], axis=1); submission.head()
final_pred = pd.concat([submission1,df1], axis=1); final_pred.head()
final_pred.to_csv(r'C:\Users\1014070\Favorites\AB\TG\HCP-June23\M9.csv',index=False)
print("Process Completed")

Accuracy of Linear SVC on training set: 0.70
Accuracy of Linear SVC on test set: 0.70
---------------------------------------------------------------------------------------------------------------------
Classification Report of the training data:

               precision    recall  f1-score   support

         0.0       0.70      1.00      0.82     19950
         1.0       0.00      0.00      0.00      8535

    accuracy                           0.70     28485
   macro avg       0.35      0.50      0.41     28485
weighted avg       0.49      0.70      0.58     28485
 

<class 'pandas.core.frame.DataFrame'>
Process Completed


In [13]:
ab = make_pipeline(StandardScaler(),SGDClassifier(loss = 'hinge')) 
#model10 = CalibratedClassifierCV(ab) #45.5708 S1
#model10 = CalibratedClassifierCV(ab) #45.7237 S2
#model10 = CalibratedClassifierCV(ab) #45.5743 S3


model10 = CalibratedClassifierCV(ab) #41.277 S5
model10.fit(X_train,y_train) 
print('Accuracy of SGD Classifier on training set: {:.2f}'
     .format(model10.score(X_train, y_train)))

print('Accuracy of SGD Classifier on test set: {:.2f}'
     .format(model10.score(X_test, y_test)))
print('-------------'*9)
print('Classification Report of the training data:\n\n',classification_report(y_test,model10.predict(X_test)),'\n')

# Import Test data for the prediction of the Target Variable
x = np.array(test)

# Prediction using above Tuned Mode

y_pred10 = model10.predict(x); 
y_pred = y_pred10

# Save in Dataframe
df1=pd.DataFrame(y_pred,columns=['IS_HCP']); print(type(df1))

submission.head(); submission1 = submission.drop(['IS_HCP'], axis=1); submission.head()
final_pred = pd.concat([submission1,df1], axis=1); final_pred.head()
final_pred.to_csv(r'C:\Users\1014070\Favorites\AB\TG\HCP-June23\M10.csv',index=False)
print("Process Completed")

Accuracy of SGD Classifier on training set: 0.70
Accuracy of SGD Classifier on test set: 0.70
---------------------------------------------------------------------------------------------------------------------
Classification Report of the training data:

               precision    recall  f1-score   support

         0.0       0.70      1.00      0.82     19950
         1.0       0.00      0.00      0.00      8535

    accuracy                           0.70     28485
   macro avg       0.35      0.50      0.41     28485
weighted avg       0.49      0.70      0.58     28485
 

<class 'pandas.core.frame.DataFrame'>
Process Completed
